In [1]:
import os
import sys
import time
import numpy as np
import scipy.sparse as sp
from keras.layers import Input, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2

Using TensorFlow backend.


In [2]:
# the path of execution
EXE_PATH = os.path.abspath(os.path.curdir)
# the path of the vendor files
VENDOR_PATH = os.path.join(EXE_PATH, 'vendor')
# the vendors to include in the system path
VENDORS = ['keras-gcn']
# create the absolute paths for all vendors
VENDORS = list(map(lambda x: os.path.join(VENDOR_PATH, x), VENDORS))
# update the Python path to include necessary vendor module
sys.path += VENDORS

In [3]:
from kegra.layers.graph import GraphConvolution
from kegra.utils import load_data, preprocess_adj, get_splits, evaluate_preds
from ggcn import GaussianGraphConvolution

# Data

In [4]:
X, A, y = load_data('data/cora/', dataset='cora')
X /= X.sum(1).reshape(-1, 1)
A = preprocess_adj(A)
y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask = get_splits(y)

Loading cora dataset...
Dataset has 2708 nodes, 5429 edges, 1433 features.


In [5]:
X

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [6]:
A

<2708x2708 sparse matrix of type '<class 'numpy.float64'>'
	with 13264 stored elements in Compressed Sparse Row format>

In [7]:
y

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0]], dtype=int32)

In [8]:
support = 1
graph = [X, A]
G = [Input(shape=(None, None), batch_shape=(None, None), sparse=True)]

W1110 16:44:54.339621 140535297824576 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1110 16:44:54.346390 140535297824576 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:515: The name tf.sparse_placeholder is deprecated. Please use tf.compat.v1.sparse_placeholder instead.



In [9]:
X_in = Input(shape=(X.shape[1],))

W1110 16:44:54.351480 140535297824576 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



# GCN (baseline)

In [10]:
# Define model architecture
# NOTE: We pass arguments for graph convolutional layers as a list of tensors.
# This is somewhat hacky, more elegant options would require rewriting the Layer base class.
H = Dropout(0.5)(X_in)
H = GraphConvolution(16, support, activation='relu', kernel_regularizer=l2(5e-4))([H]+G)
H = Dropout(0.5)(H)
Y = GraphConvolution(y.shape[1], support, activation='softmax')([H]+G)

W1110 16:44:54.356712 140535297824576 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W1110 16:44:54.361202 140535297824576 deprecation.py:506] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W1110 16:44:54.369642 140535297824576 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [11]:
model = Model(inputs=[X_in]+G, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))

W1110 16:44:54.411627 140535297824576 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [12]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1433)         0                                            
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 1433)         0           input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
graph_convolution_1 (GraphConvo (None, 16)           22944       dropout_1[0][0]                  
                                                                 input_1[0][0]                    
__________

In [13]:
NB_EPOCH = 200
PATIENCE = 10  # early stopping patience
# Helper variables for main training loop
wait = 0
preds = None
best_val_loss = 99999

# Fit
for epoch in range(1, NB_EPOCH+1):

    # Log wall-clock time
    t = time.time()

    # Single training iteration (we mask nodes without labels for loss calculation)
    model.fit(graph, y_train, sample_weight=train_mask,
              batch_size=A.shape[0], epochs=1, shuffle=False, verbose=0)

    # Predict on full dataset
    preds = model.predict(graph, batch_size=A.shape[0])

    # Train / validation scores
    train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val],
                                                   [idx_train, idx_val])
    print("Epoch: {:04d}".format(epoch),
          "train_loss= {:.4f}".format(train_val_loss[0]),
          "train_acc= {:.4f}".format(train_val_acc[0]),
          "val_loss= {:.4f}".format(train_val_loss[1]),
          "val_acc= {:.4f}".format(train_val_acc[1]),
          "time= {:.4f}".format(time.time() - t))

    # Early stopping
    if train_val_loss[1] < best_val_loss:
        best_val_loss = train_val_loss[1]
        wait = 0
    else:
        if wait >= PATIENCE:
            print('Epoch {}: early stopping'.format(epoch))
            break
        wait += 1

W1110 16:44:54.489424 140535297824576 deprecation.py:323] From /home/bitcommander/.local/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch: 0001 train_loss= 1.9363 train_acc= 0.4357 val_loss= 1.9380 val_acc= 0.3600 time= 1.0612
Epoch: 0002 train_loss= 1.9262 train_acc= 0.3357 val_loss= 1.9284 val_acc= 0.3667 time= 0.0240
Epoch: 0003 train_loss= 1.9152 train_acc= 0.3357 val_loss= 1.9187 val_acc= 0.3633 time= 0.0284
Epoch: 0004 train_loss= 1.9035 train_acc= 0.3286 val_loss= 1.9084 val_acc= 0.3600 time= 0.0251
Epoch: 0005 train_loss= 1.8914 train_acc= 0.3143 val_loss= 1.8978 val_acc= 0.3567 time= 0.0257
Epoch: 0006 train_loss= 1.8792 train_acc= 0.3143 val_loss= 1.8871 val_acc= 0.3567 time= 0.0240
Epoch: 0007 train_loss= 1.8668 train_acc= 0.3143 val_loss= 1.8763 val_acc= 0.3567 time= 0.0282
Epoch: 0008 train_loss= 1.8541 train_acc= 0.3143 val_loss= 1.8655 val_acc= 0.3567 time= 0.0244
Epoch: 0009 train_loss= 1.8415 train_acc= 0.3143 val_loss= 1.8547 val_acc= 0.3567 time= 0.0285
Epoch: 0010 train_loss= 1.8288 train_acc= 0.3214 val_loss= 1.8440 val_acc= 0.3533 time= 0.0194
Epoch: 0011 train_loss= 1.8163 train_acc= 0.3143 v

Epoch: 0089 train_loss= 1.0384 train_acc= 0.7929 val_loss= 1.2754 val_acc= 0.6733 time= 0.0218
Epoch: 0090 train_loss= 1.0313 train_acc= 0.7929 val_loss= 1.2708 val_acc= 0.6733 time= 0.0253
Epoch: 0091 train_loss= 1.0239 train_acc= 0.7929 val_loss= 1.2658 val_acc= 0.6767 time= 0.0245
Epoch: 0092 train_loss= 1.0163 train_acc= 0.7929 val_loss= 1.2610 val_acc= 0.6833 time= 0.0232
Epoch: 0093 train_loss= 1.0086 train_acc= 0.7929 val_loss= 1.2555 val_acc= 0.6867 time= 0.0182
Epoch: 0094 train_loss= 1.0010 train_acc= 0.8000 val_loss= 1.2498 val_acc= 0.6867 time= 0.0184
Epoch: 0095 train_loss= 0.9937 train_acc= 0.8000 val_loss= 1.2439 val_acc= 0.6867 time= 0.0227
Epoch: 0096 train_loss= 0.9868 train_acc= 0.8000 val_loss= 1.2386 val_acc= 0.6867 time= 0.0186
Epoch: 0097 train_loss= 0.9802 train_acc= 0.8000 val_loss= 1.2342 val_acc= 0.7033 time= 0.0278
Epoch: 0098 train_loss= 0.9736 train_acc= 0.7929 val_loss= 1.2290 val_acc= 0.7000 time= 0.0227
Epoch: 0099 train_loss= 0.9669 train_acc= 0.7929 v

Epoch: 0180 train_loss= 0.6070 train_acc= 0.9071 val_loss= 0.9522 val_acc= 0.7667 time= 0.0237
Epoch: 0181 train_loss= 0.6040 train_acc= 0.9000 val_loss= 0.9492 val_acc= 0.7667 time= 0.0208
Epoch: 0182 train_loss= 0.6011 train_acc= 0.9000 val_loss= 0.9464 val_acc= 0.7733 time= 0.0232
Epoch: 0183 train_loss= 0.5978 train_acc= 0.9000 val_loss= 0.9443 val_acc= 0.7767 time= 0.0251
Epoch: 0184 train_loss= 0.5944 train_acc= 0.9000 val_loss= 0.9423 val_acc= 0.7767 time= 0.0206
Epoch: 0185 train_loss= 0.5909 train_acc= 0.9000 val_loss= 0.9411 val_acc= 0.7767 time= 0.0229
Epoch: 0186 train_loss= 0.5879 train_acc= 0.9000 val_loss= 0.9419 val_acc= 0.7833 time= 0.0237
Epoch: 0187 train_loss= 0.5855 train_acc= 0.9000 val_loss= 0.9407 val_acc= 0.7833 time= 0.0232
Epoch: 0188 train_loss= 0.5833 train_acc= 0.9000 val_loss= 0.9393 val_acc= 0.7767 time= 0.0288
Epoch: 0189 train_loss= 0.5812 train_acc= 0.9000 val_loss= 0.9381 val_acc= 0.7767 time= 0.0235
Epoch: 0190 train_loss= 0.5791 train_acc= 0.9000 v

In [14]:
test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
print(f"""
loss = {test_loss[0]:.4f}
accuracy = {test_acc[0]:.4f}
""".strip())

loss = 0.9675
accuracy = 0.7370


# Gaussian Graph Convolution

In [15]:
H = Dropout(0.5)(X_in)
H = GaussianGraphConvolution(16, A.shape[0], 
    is_first=True, 
    activation='relu', 
    kernel_regularizer=l2(5e-4)
)([H]+G)
Y = GaussianGraphConvolution(y.shape[1], A.shape[0], 
    is_last=True,
    activation='softmax'
)(H+G)

W1110 16:45:00.016038 140535297824576 deprecation.py:323] From /home/bitcommander/Desktop/robust-graph-convolutional-networks-against-adversarial-attacks-implementation/ggcn/ggcl.py:163: Normal.__init__ (from tensorflow.python.ops.distributions.normal) is deprecated and will be removed after 2019-01-01.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
W1110 16:45:00.017428 140535297824576 deprecation.py:323] From /home/bitcommander/.local/lib/python3.7/site-packages/tensorflow/python/ops/distributions/normal.py:160: Distribution.__init__ (from tensorflow.python.ops.distributions.distribution) is deprecated and will be removed after 2019-01-01.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should updat

In [16]:
model = Model(inputs=[X_in]+G, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))

In [17]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1433)         0                                            
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 1433)         0           input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
gaussian_graph_convolution_1 (G [(None, 16), (None,  45856       dropout_3[0][0]                  
                                                                 input_1[0][0]                    
__________

In [18]:
NB_EPOCH = 200
PATIENCE = 10  # early stopping patience
# Helper variables for main training loop
wait = 0
preds = None
best_val_loss = 99999

# Fit
for epoch in range(1, NB_EPOCH+1):

    # Log wall-clock time
    t = time.time()

    # Single training iteration (we mask nodes without labels for loss calculation)
    model.fit(graph, y_train, sample_weight=train_mask,
              batch_size=A.shape[0], epochs=1, shuffle=False, verbose=0)

    # Predict on full dataset
    preds = model.predict(graph, batch_size=A.shape[0])

    # Train / validation scores
    train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val],
                                                   [idx_train, idx_val])
    print("Epoch: {:04d}".format(epoch),
          "train_loss= {:.4f}".format(train_val_loss[0]),
          "train_acc= {:.4f}".format(train_val_acc[0]),
          "val_loss= {:.4f}".format(train_val_loss[1]),
          "val_acc= {:.4f}".format(train_val_acc[1]),
          "time= {:.4f}".format(time.time() - t))

    # Early stopping
    if train_val_loss[1] < best_val_loss:
        best_val_loss = train_val_loss[1]
        wait = 0
    else:
        if wait >= PATIENCE:
            print('Epoch {}: early stopping'.format(epoch))
            break
        wait += 1

Epoch: 0001 train_loss= 1.9396 train_acc= 0.3786 val_loss= 1.9416 val_acc= 0.3300 time= 0.4114
Epoch: 0002 train_loss= 1.9315 train_acc= 0.5786 val_loss= 1.9362 val_acc= 0.4233 time= 0.0182
Epoch: 0003 train_loss= 1.9222 train_acc= 0.5929 val_loss= 1.9295 val_acc= 0.4733 time= 0.0227
Epoch: 0004 train_loss= 1.9107 train_acc= 0.6714 val_loss= 1.9213 val_acc= 0.5467 time= 0.0228
Epoch: 0005 train_loss= 1.8989 train_acc= 0.7143 val_loss= 1.9131 val_acc= 0.5500 time= 0.0179
Epoch: 0006 train_loss= 1.8871 train_acc= 0.7500 val_loss= 1.9040 val_acc= 0.5967 time= 0.0236
Epoch: 0007 train_loss= 1.8726 train_acc= 0.7429 val_loss= 1.8928 val_acc= 0.6400 time= 0.0224
Epoch: 0008 train_loss= 1.8602 train_acc= 0.7286 val_loss= 1.8845 val_acc= 0.6200 time= 0.0190
Epoch: 0009 train_loss= 1.8432 train_acc= 0.7286 val_loss= 1.8729 val_acc= 0.6300 time= 0.0232
Epoch: 0010 train_loss= 1.8301 train_acc= 0.7000 val_loss= 1.8615 val_acc= 0.6267 time= 0.0225
Epoch: 0011 train_loss= 1.8125 train_acc= 0.7429 v

Epoch: 0093 train_loss= 0.2972 train_acc= 0.9357 val_loss= 0.6936 val_acc= 0.7967 time= 0.0208
Epoch: 0094 train_loss= 0.2909 train_acc= 0.9357 val_loss= 0.6915 val_acc= 0.8000 time= 0.0197
Epoch: 0095 train_loss= 0.2854 train_acc= 0.9357 val_loss= 0.6862 val_acc= 0.8000 time= 0.0188
Epoch: 0096 train_loss= 0.2805 train_acc= 0.9357 val_loss= 0.6820 val_acc= 0.7967 time= 0.0231
Epoch: 0097 train_loss= 0.2746 train_acc= 0.9357 val_loss= 0.6797 val_acc= 0.8000 time= 0.0184
Epoch: 0098 train_loss= 0.2683 train_acc= 0.9357 val_loss= 0.6746 val_acc= 0.8000 time= 0.0185
Epoch: 0099 train_loss= 0.2616 train_acc= 0.9357 val_loss= 0.6724 val_acc= 0.8000 time= 0.0279
Epoch: 0100 train_loss= 0.2564 train_acc= 0.9357 val_loss= 0.6691 val_acc= 0.8000 time= 0.0242
Epoch: 0101 train_loss= 0.2534 train_acc= 0.9357 val_loss= 0.6658 val_acc= 0.8000 time= 0.0190
Epoch: 0102 train_loss= 0.2484 train_acc= 0.9357 val_loss= 0.6634 val_acc= 0.8033 time= 0.0273
Epoch: 0103 train_loss= 0.2430 train_acc= 0.9357 v

In [19]:
test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
print(f"""
loss = {test_loss[0]:.4f}
accuracy = {test_acc[0]:.4f}
""".strip())

loss = 0.6044
accuracy = 0.8120


# Attacks